In [1]:
# %matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import torch
import random
from torch import nn
from torch.nn import functional as F
import os
from pathlib import Path
import numpy as np
import torch.optim as optim
from torchvision.transforms.transforms import RandomApply
from core.transforms import GaussianBlur, TwoCropsTransform
from core.custom_dataset import DatasetFolderSorted, ImageFolderSorted
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Dataset
from agents.net import MLP, Encoder, SimSiam
import time

In [2]:

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
augmentation = [
    transforms.RandomResizedCrop(50, scale=(0.2, 1.)),
    transforms.RandomApply([
        transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)  # not strengthened
    ], p=0.8),
    transforms.RandomGrayscale(p=0.2),
    transforms.RandomApply([GaussianBlur([.1, 2.])], p=0.5),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize
]

In [3]:
train_dataset = datasets.ImageFolder(
    'dataset/MiniWorld-Hallway-v0/agent/D250',
    TwoCropsTransform(
        transforms.Compose(augmentation)
    )
)

In [4]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=256,
    shuffle=True
)

In [5]:
model = SimSiam().cuda()


In [6]:
criterion = nn.CosineSimilarity(dim=1).cuda()
fix_pred_lr = False
init_lr = 0.001

if fix_pred_lr:
    optim_params = [{'params': model.module.encoder.parameters(), 'fix_lr': False},
                    {'params': model.module.predictor.parameters(), 'fix_lr': True}]
else:
    optim_params = model.parameters()

optimizer = optim.Adam(optim_params, lr=0.05)

In [7]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)


class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

In [8]:
def train(train_loader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4f')
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses],
        prefix="Epoch: [{}]".format(epoch))

    # switch to train mode
    model.train()

    end = time.time()
    for i, (images, _) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        # if args.gpu is not None:
        #     images[0] = images[0].cuda(args.gpu, non_blocking=True)
        #     images[1] = images[1].cuda(args.gpu, non_blocking=True)

        # compute output and loss
        # print(len(images))
        # input()
        p1, p2, z1, z2 = model(x1=images[0].cuda(), x2=images[1].cuda())
        # print(p1.size(), p2.size(), z1.size(), z2.size())
        loss = -(criterion(p1, z2).mean() + criterion(p2, z1).mean()) * 0.5

        losses.update(loss.item(), images[0].size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % (max(len(train_loader)//10, 1)) == 0:
            progress.display(i)
for epoch in range(1, 1 + 10):
    train(train_loader, model, criterion, optimizer, epoch=epoch)

Epoch: [1][ 0/36]	Time  0.566 ( 0.566)	Data  0.384 ( 0.384)	Loss 0.0981 (0.0981)
Epoch: [1][ 3/36]	Time  0.379 ( 0.424)	Data  0.372 ( 0.373)	Loss -0.6236 (-0.3508)
Epoch: [1][ 6/36]	Time  0.365 ( 0.401)	Data  0.356 ( 0.368)	Loss -0.8106 (-0.5276)
Epoch: [1][ 9/36]	Time  0.363 ( 0.389)	Data  0.354 ( 0.364)	Loss -0.8888 (-0.6291)
Epoch: [1][12/36]	Time  0.350 ( 0.382)	Data  0.343 ( 0.361)	Loss -0.9343 (-0.6966)
Epoch: [1][15/36]	Time  0.359 ( 0.378)	Data  0.352 ( 0.359)	Loss -0.9605 (-0.7446)
Epoch: [1][18/36]	Time  0.362 ( 0.374)	Data  0.354 ( 0.357)	Loss -0.9744 (-0.7803)
Epoch: [1][21/36]	Time  0.355 ( 0.373)	Data  0.348 ( 0.358)	Loss -0.9825 (-0.8076)
Epoch: [1][24/36]	Time  0.367 ( 0.372)	Data  0.359 ( 0.357)	Loss -0.9871 (-0.8289)
Epoch: [1][27/36]	Time  0.374 ( 0.372)	Data  0.365 ( 0.358)	Loss -0.9905 (-0.8461)
Epoch: [1][30/36]	Time  0.378 ( 0.372)	Data  0.370 ( 0.358)	Loss -0.9927 (-0.8603)
Epoch: [1][33/36]	Time  0.362 ( 0.372)	Data  0.354 ( 0.359)	Loss -0.9946 (-0.8721)
Epoch: